In [1]:
!nvcc --version

/bin/bash: -c: line 2: syntax error: unexpected end of file


In [2]:
!pip install nvcc4jupyter


In [3]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp4_37zb9n".


In [4]:
%%cuda
#include <iostream>
#include <vector>
#include <limits.h>
#include <chrono>
#include <bitset>

#define MAX_V 4  // maximum number of nodes

const int dist[MAX_V][MAX_V] = {
    { 0, 10, 15, 20 },
    { 10, 0, 35, 25 },
    { 15, 35, 0, 30 },
    { 20, 25, 30, 0 }
};

// The main dynamic programming function for solving TSP
int tspDP(int mask, int pos, std::vector<std::vector<int>>& dp, int n) {
    if (mask == ((1 << n) - 1)) {
        return dist[pos][0]; // Return to the starting city (city 0)
    }

    if (dp[mask][pos] != -1) {
        return dp[mask][pos];
    }

    int ans = INT_MAX;
    for (int city = 0; city < n; city++) {
        if ((mask & (1 << city)) == 0) {
            int newAns = dist[pos][city] + tspDP(mask | (1 << city), city, dp, n);
            ans = std::min(ans, newAns);
        }
    }

    return dp[mask][pos] = ans;
}

int main() {
    int n = MAX_V;  // number of cities

    // Initialize DP table
    std::vector<std::vector<int>> dp(1 << n, std::vector<int>(n, -1));

    // Measure execution time
    auto start = std::chrono::high_resolution_clock::now();

    // Start TSP from node 0 and mask 1 (visited node 0)
    int result = tspDP(1, 0, dp, n);

    auto end = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> duration = end - start;

    std::cout << "Nodes: " << n << " Time: " << duration.count() << " seconds. Min Cost: " << result << "\n";

    // Print the optimal tour
    int mask = 1, pos = 0;
    std::cout << "Optimal Tour: 0";
    for (int i = 0; i < n - 1; i++) {
        int nextCity = -1;
        for (int city = 0; city < n; city++) {
            if ((mask & (1 << city)) == 0) {
                if (nextCity == -1 || dist[pos][city] + dp[mask | (1 << city)][city] <
                                      dist[pos][nextCity] + dp[mask | (1 << nextCity)][nextCity]) {
                    nextCity = city;
                }
            }
        }
        std::cout << " -> " << nextCity;
        mask |= (1 << nextCity);
        pos = nextCity;
    }
    std::cout << " -> 0\n";

    return 0;
}


Nodes: 4 Time: 1.173e-06 seconds. Min Cost: 80
Optimal Tour: 0 -> 1 -> 3 -> 2 -> 0



In [5]:
%%cuda
#include <iostream>
#include <vector>
#include <limits.h>
#include <chrono>
#include <random>

#define MAX 20  // maximum number of nodes

std::vector<std::vector<int>> generateRandomAdjacencyMatrix(int nodes, int maxWeight) {
    std::random_device rd;
    std::mt19937 gen(rd());
    std::uniform_int_distribution<> dis(1, maxWeight);

    std::vector<std::vector<int>> matrix(nodes, std::vector<int>(nodes));
    for (int i = 0; i < nodes; ++i) {
        for (int j = 0; j < nodes; ++j) {
            if (i == j) {
                matrix[i][j] = 0;  // Diagonal elements are zero (no self-loop)
            } else {
                matrix[i][j] = dis(gen);
            }
        }
    }
    return matrix;
}

// The main dynamic programming function for solving TSP
int tspDP(int mask, int pos, const std::vector<std::vector<int>>& adj, std::vector<std::vector<int>>& dp, int n) {
    if (mask == (1 << n) - 1) {
        return adj[pos][0]; // Return to the starting city
    }

    if (dp[mask][pos] != -1) {
        return dp[mask][pos];
    }

    int ans = INT_MAX;
    for (int city = 0; city < n; city++) {
        if ((mask & (1 << city)) == 0) {
            int newAns = adj[pos][city] + tspDP(mask | (1 << city), city, adj, dp, n);
            ans = std::min(ans, newAns);
        }
    }

    return dp[mask][pos] = ans;
}

int main() {
    int maxWeight = 100;

    for (int nodes = 4; nodes <= MAX; nodes++) {
        // Generate random adjacency matrix on CPU
        auto adj = generateRandomAdjacencyMatrix(nodes, maxWeight);

        // Debug: Print the random matrix
        //std::cout << "Adjacency Matrix (" << nodes << "x" << nodes << "):\n";
        int zeroCount = 0;
        //for (int i = 0; i < nodes; i++) {
           // for (int j = 0; j < nodes; j++) {
              //  std::cout << adj[i][j] << " ";
               // if (adj[i][j] == 0) zeroCount++;
           // }
           // std::cout << "\n";
        //}
        //std::cout << "Number of zeros: " << zeroCount << " (expected: " << nodes << ")\n\n";

        // Initialize DP table
        std::vector<std::vector<int>> dp(1 << nodes, std::vector<int>(nodes, -1));

        // Measure execution time
        auto start = std::chrono::high_resolution_clock::now();

        // Start TSP from node 0 and mask 1 (visited node 0)
        int result = tspDP(1, 0, adj, dp, nodes);

        auto end = std::chrono::high_resolution_clock::now();
        std::chrono::duration<double> duration = end - start;

        std::cout << "Nodes: " << nodes << " Time: " << duration.count() << " seconds. Cost: " << result << "\n";
    }

    return 0;
}







Nodes: 4 Time: 1.023e-06 seconds. Cost: 136
Nodes: 5 Time: 2.773e-06 seconds. Cost: 184
Nodes: 6 Time: 7.472e-06 seconds. Cost: 209
Nodes: 7 Time: 2.0772e-05 seconds. Cost: 321
Nodes: 8 Time: 5.479e-05 seconds. Cost: 221
Nodes: 9 Time: 0.000254195 seconds. Cost: 131
Nodes: 10 Time: 0.000346224 seconds. Cost: 142
Nodes: 11 Time: 0.000860431 seconds. Cost: 99
Nodes: 12 Time: 0.00205667 seconds. Cost: 189
Nodes: 13 Time: 0.00478523 seconds. Cost: 172
Nodes: 14 Time: 0.0114477 seconds. Cost: 147
Nodes: 15 Time: 0.0265484 seconds. Cost: 116
Nodes: 16 Time: 0.0650195 seconds. Cost: 188
Nodes: 17 Time: 0.164437 seconds. Cost: 146
Nodes: 18 Time: 0.375428 seconds. Cost: 149
Nodes: 19 Time: 0.998055 seconds. Cost: 211
Nodes: 20 Time: 2.504 seconds. Cost: 144

